# Sort the ARD images into ice/water/mix classes
This code takes all the processed images from the main SeaIceCNN folder and sorts them into three classes (ice/water/mix). They are all copied into respective subfolders of "Data" in order to facilitate the training of the CNN model (this is really useful in the case of batch training, Keras has tools to reads batches of subfolders)

In [1]:
#Importing start/end dates for ice/water periods
import pandas as pd
dates = pd.read_excel(r"Data\IceWaterDates.xlsx")

In [4]:
#Reading all the files in the main folder
from os import listdir, remove
files = [f for f in listdir() if f.endswith('.tif') and f.startswith('S0_RS2_')]
print(files)

['S0_RS2_OK9968_PK112693_DK109010_SCWA_20100213_004158_HH_HV_SGF.tif']


In [6]:
#Importing libraries
from datetime import datetime
from shutil import copyfile
from os.path import join, isfile, isdir
from os import mkdir

In [7]:
#Making the subfolders if they don't exist
if not isdir('Data/ice'):
    mkdir('Data/ice')
    
if not isdir('Data/water'):
    mkdir('Data/water')
    
if not isdir('Data/mix'):
    mkdir('Data/mix')

In [9]:
#Sorts the files based on the acquisitions dates
for f in files:
    
    #Read acquisition date of the image from the filename
    year = int(f.split('_')[6][0:4])
    month = int(f.split('_')[6][4:6])
    day = int(f.split('_')[6][6:8])
    date = datetime(year, month, day)
    
    #Initiate an index to determine if the file is a training data file
    x = 0
    
    #For each start/end period of every year
    for y in range(len(dates)):
        
        #If the date is within the ice free (water) period
        if (date>=dates.iloc[y,0]) & (date<=dates.iloc[y,1]):
            
            #Simply remove the file if it already exists in the subfolder
            if isfile(join(r'Data', 'water', f)):
                
                remove(f)
            
            #Copy the file to the subfolder and delete it from the main folder
            else:
                copyfile(f, join(r'Data', 'water', f))
                remove(f)
                #File was copied, change the index
            x = 1
            
        #If the date is within the ice covered period
        elif (date>=dates.iloc[y,2]) & (date<=dates.iloc[y,3]):
            
            #Simply remove the file if it already exists in the subfolder
            if isfile(join(r'Data', 'ice', f)):
                
                remove(f)
                
            #Copy the file to the subfolder and delete it from the main folder
            else:
                
                copyfile(f, join(r'Data', 'ice', f))
                remove(f)
                #File was copied, change the index
            x = 1
    
    #If the file was not copied in training folders, i.e. it's not complete ice or water, 
    #copy the file in the "mix" folder
    if x == 0:
        
        #Simply remove the file if it already exists in the subfolder
        if isfile(join(r'Data', 'mix', f)):
                
                remove(f)
           
        #Copy the file to the subfolder and delete it from the main folder
        else:
            
            copyfile(f, join(r'Data', 'mix', f))
            remove(f)